In [43]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import sinh, cosh

In [17]:
# Initial matrixes A x = f:
# a -> A
# b -> f

matrix_a = [
    [4.0, 1.0, 0.0, 0.0],
    [1.0, 4.0, 1.0, 0.0],
    [0.0, 1.0, 4.0, 1.0],
    [0.0, 0.0, 1.0, 4.0]
]
matrix_a = np.array(matrix_a)

In [18]:
matrix_b = [6.0, 12.0, 18.0, 19.0]
matrix_b = np.array(matrix_b)

In [19]:
# The solution using the python function
np.linalg.solve(matrix_a, matrix_b)

array([1., 2., 3., 4.])

In [20]:
# Function to find the coefficients of a linear system of the form A x = f.
# Where A is a tridiagonal matrix, f is a vector with the function values,
# and x is a vector which we want to find its components to solve the system.

def solution_coefficients(matrix_A, vector_f):
    matrix = matrix_A.copy()
    vector = vector_f.copy()
    
    for i in range(1, np.shape(matrix)[0]):
        vector[i] = vector[i] - (matrix[i, i-1] / matrix[i-1, i-1]) * vector[i-1]
        matrix[i] = matrix[i] - (matrix[i, i-1] / matrix[i-1, i-1]) * matrix[i-1]
    
    coefficients = np.array([vector[-1] / matrix[-1, -1]])
    
    for i in range(np.shape(matrix)[0] - 2, -1, -1):
        x = (vector[i] - matrix[i, i+1] * coefficients[0]) / matrix[i, i]
        coefficients = np.insert(coefficients, 0, x)
        
    return coefficients

In [21]:
# The order of the coefficients is as follows:
# sol = {sol_0, sol_1, sol_2, ... , sol_n}

solution_coefficients(matrix_a, matrix_b)

array([1., 2., 3., 4.])

Now, I shall generate the matrix to find the $z_{i}$ coefficients for the tension spline. With the funciton defined above *solution_coefficients* I can solve the resulting system of equations.

In [22]:
nodes_x = [0.0, 0.9, 1.8, 8.3, 11.0, 18.0, 20.4, 23.6, 25.0]
nodes_y = [1.3, 1.3, 2.9, 3.8, 7.2, 7.2, 4.6, 4.6, 0.0]
tau = 10

In [44]:
# MUST CHECK THE DEFINITION for h here. There is something odd with the indexes. 
def h(i):
    return nodes_x[i+1] - nodes_x[i]

In [45]:
def alpha(i):
    return (1 / h(i)) - (tau / sinh(tau * h(i)))

In [48]:
def beta(i):
    return (tau * cosh(tau * h(i)) / sinh(tau * h(i))) - (1 / h(i))

In [50]:
def gamma(i):
    return tau**2 * (nodes_y[i+1] - nodes_y[i]) / h(i)